# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [4]:
# Imagen para estilo
#!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

#!wget https://commons.wikimedia.org/wiki/File:Guernica_Rathaus_PF.jpg#/media/File:Guernica_Rathaus_PF.jpg

#!wget https://upload.wikimedia.org/wikipedia/commons/c/cc/Juan_Miró%2C_Landscape_%28The_Grasshopper%29_1926%3B_oil%2C_Goulandris_Museum%2C_Athens.jpg
# Imagen para contenido
#!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

#!wget https://upload.wikimedia.org/wikipedia/en/4/4c/Les_Demoiselles_d%27Avignon.jpg

# Creamos el directorio para los archivos de salida
#!mkdir /content/output

--2020-06-20 19:08:55--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg.1’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.02s   

2020-06-20 19:08:55 (13.4 MB/s) - ‘La_noche_estrellada1.jpg.1’ saved [223725/223725]

--2020-06-20 19:08:57--  https://commons.wikimedia.org/wiki/File:Guernica_Rathaus_PF.jpg
Resolving commons.wikimedia.org (commons.wikimedia.org)... 91.198.174.192, 2620:0:861:ed1a::1
Connecting to commons.wikimedia.org (commons.wikimedia.org)|91.198.174.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘File:Guernica_Rathaus_PF.jpg.1’

File:Guernica_Ratha  

In [27]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

In [28]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

!mkdir /output
result_prefix = Path("/output")


base_image_path = Path("/input/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/input/La_noche_estrellada1.jpg")

#definimos paths para la pregunta 9
style_reference_image_path_2 = Path("/input/Les_Demoiselles_d'Avignon.jpg")
base_image_path_2 = Path("/input/sol_2.jpg")


# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

**Respuesta:**

En base a lo leido en el paper, se entiende que la loss a minimizar para la generación de las imagenes, se encuentra definido por 2 tipos de losses: una de estilo (style) y otra de contenido (content). Los parametros *Style Weight* y *Content Weight* son los pesos que tiene cada loss. Cuando se pone un peso muy grande en Content Weight, se puede identificar bien la fotografia, pero el estilo de la pintura no coinside. Con lo cual existe un trade-off entre cada una de las losses y por ende de los pesos. 
Dentro del calculo de la loss por estilo, el *Total Variation Weight* es la contribucion de cada capa a la loss total. 



In [29]:
iterations = 100
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [31]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

**Respuesta:**
La funcion lo que hace es:


1. Carga la imagen y define un formato de target
2. Transforma la imagen al formato de Python array (tensor)
3. Agrega una dimensión al comienzo del array (tensor) de numpy. porque? Porque las redes neuronales esperan que la primer dimension del array (tensor) corresponda al nro de batches. 
4. Aplica el modelo pre-entrenado por ImageNet y aplica el comando preprocess?input que lo que hace es convertir el tensor en una imagen valida. La funcion recive un array y lo que hace es convertir la imagen de RGB a VGR y centra cada color en 0 con respecto a la base de datos de ImageNet. 




In [32]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

**Respuesta:** la funcion lo que hace es volver a procer la iamgen para transformala con el formato original. Por lo tanto, se le resetea el centro de los colores para que deje de estar centrado en 0 y se pasa la medición de colores de BGR a RGB.

La funcion primero hace un reshape de la imagen, para luego sumarle los escalares para que los colores dejen de estar 0 centrados. Luego pasa primer dimención del canal BGR a la ultima posicion y la ultima posicion a la primera, para quedar en RGB


In [33]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [34]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [35]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [36]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [37]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

Model loaded.


# 4) En la siguientes celdas:

- **¿Qué es la matriz de Gram?¿Para qué se usa?** La matriz Gram muestra la correlacion entre los diferentes filtros de una capa. Se utiliza para calcular la contribucion de cada capa a la loss de estilo. Es decir, al minimizar las distancias cuadradas medias de las entradas de la matriz Gram de la imagen original  y de la matriz Gram de la imagen generada se obtiene la contribucion de cada capa a la loss de estilo. 
- **¿Por qué se permutan las dimensiones de x?** Se permutan las dimensiones del vector de entrada de la matriz Gram para que queden en el orden RGB.

In [38]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

**Respuesta:**


*   **Style Loss:** Mide la diferencia entre la imagen original y la imagen creada con estilos en cada capa. Esta loss se calcula por medio de la minimización de la distancia media al cuadrado de las matrices de Gram (uso de correlaciones) para cada una de las imagenes. Esta loss se encuentra afectada por un factor que indica cuanto afecta cada estilo a la loss total. 
*  **Content Loss:** Mide el error cuadratico medio entre la imagen original y la imagen generada en cada capa. En esta loss lo que se busca hacer es un Content Reconstructions, donde se recontruye la imagen orginal utilizando solamente la respuesta de la red en las diferentes capas. 
*  **Total Variation Loss:** Es la combinacion lineal de las 2 losses (Style y Content Loss) multiplicado por 2 factores (weights) alfa y beta que ponderan el peso de cada una de las losses.


In [39]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [40]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [41]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [42]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                              combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                    'block3_conv1', 'block4_conv1',
                    'block5_conv1']
for layer_name in feature_layers:
  layer_features = outputs_dict[layer_name]
  style_reference_features = layer_features[1, :, :, :] 
  combination_features = layer_features[2, :, :, :]
  sl = style_loss(style_reference_features, combination_features)
  loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [43]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

**Respuesta:**

La celda que comienza con *class Evaluator(object)* define las funciones dentro de la clase Evaluator para obtener los resultados de la loss y los valores de los gradientes. 

La celda que comienza con *def eval_loss_and_grads(x):*  toma en base a un input X la loss y los gradientes.

La funcion *fmin_l_bfgs_b* lo que hace es minimizar una funcion por medio de la implementacion del algoritmo L-BFGS-B.

In [44]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [45]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

<img src="images\pregunta_7.jpeg">


Como se puede ver en la imagen adjunta, a medida que se incrementan las iteraciones el algoritmo logra reducir la loss y ajustar la imagen al estilo del cuadro. 

En la primera iteración se ven algunas modificaciones en las lineas, y a medida que aumentan las interacion se observa la predominancia del estilo del cuadro. Notemos que los colores cambian y se acentuan los cielos y estilo caracteristico de los cuadros de Van Gogh. 

Recordemos que los pesos de la loss de estilo es de 10 mientras que de la loss de contenido se encuentre en 1.



In [ ]:
!mkdir /output_7
result_prefix = Path("/output_7")


evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Start of iteration 0
Current loss value: 13279711000.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_7/output_at_iteration_0.png
Iteration 0 completed in 3s
Start of iteration 1
Current loss value: 6407108600.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_7/output_at_iteration_1.png
Iteration 1 completed in 3s
Start of iteration 2
Current loss value: 4333841400.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_7/output_at_iteration_2.png
Iteration 2 completed in 3s
Start of iteration 3
Current loss value: 3362596900.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_7/output_at_iteration_3.png
Iteration 3 completed in 3s
Start of iteration 4
Current loss value: 2763554000.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/sal

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta: 

Para realizar esta prueba, se iteraron los pesos de estilo y contenido sobre 3 valores 0, 1 y 100 para lograr obtener resultados bien diferenciados entre los pesos. Con el fin de simplificar el proceso, se redujo el número de interaciones en cada caso. Además, se armo una matriz para comparar los resultados obtenidos en las combinaciones propuestas tomando la utlima interacion del proceso.

<img src="images\pregunta_8.jpeg">



Cuando el peso del estilo y de contenido son igual a 0, la imagen que se genera solo se ve afectada por el factor de la loss total. Por ende, la resultante es una imagen borrosa.

Cuando el peso de estilo es 0 y se comienza a incrementar el peso de contenido, se observa que la imagen generada conserva la estructura de la fotografia. Los colores generados son mas bien pixelados y las figuras conservan bien su estructura geométrica.

En el caso en el que el peso de la estructura es igual a 0 y se comienza a incrementar el peso del estilo, se observa que la fotografia se distorsiona para adaptarse al estilo del cuadro de Van Gogh. Notemos que las lineas de las casas dejan de ser rectas para pasar a ser las lineas caracteristicas del pintor.

Cuando se varian ambos pesos, se observa que tanto los colores como la estructura de la imagen comienza a compensarse y toma tanto de la fotografía como del estilo del cuadro.




In [26]:
evaluator = Evaluator()
# Creamos el directorio para los archivos de salida
!mkdir /output_8
result_prefix = Path("/output_8")

iterations =20
# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for sw in [0,1,100]:
  style_weight = sw
  for cw in [0,1,100]:
    content_weight = cw
    for i in range(iterations):
      print('Start Style Weight', sw)
      print('Start Content Weight', cw) 
      print('Start of iteration', i)
      start_time = time.time()

      loss = K.variable(0.0)
      layer_features = outputs_dict['block5_conv2']
      base_image_features = layer_features[0, :, :, :]
      combination_features = layer_features[2, :, :, :]
      loss = loss + content_weight * content_loss(base_image_features,
                                              combination_features)
      feature_layers = ['block1_conv1', 'block2_conv1',
                    'block3_conv1', 'block4_conv1',
                    'block5_conv1']
      for layer_name in feature_layers:
        layer_features = outputs_dict[layer_name]
        style_reference_features = layer_features[1, :, :, :] 
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss = loss + (style_weight / len(feature_layers)) * sl
      loss = loss + total_variation_weight * total_variation_loss(combination_image)

      grads = K.gradients(loss, combination_image)
      outputs = [loss]
      if isinstance(grads, (list, tuple)):
        outputs += grads
      else:
        outputs.append(grads)
      f_outputs = K.function([combination_image], outputs)

      x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
      print('Current loss value:', min_val)
      # save current generated image
      img = deprocess_image(x.copy())
      fname = result_prefix / ('salida_8_iteration_%d_%d_%d.png' % (sw,cw,i))
      print('Image saved as', fname)
      save_img(fname, img)
      end_time = time.time()
      print('Image saved as', fname)
      print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start Style Weight 0
Start Content Weight 0
Start of iteration 0
Current loss value: 3080104.2
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_0.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_0.png
Iteration 0 completed in 20s
Start Style Weight 0
Start Content Weight 0
Start of iteration 1
Current loss value: 500149.4
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_1.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_1.png
Iteration 1 completed in 11s
Start Style Weight 0
Start Content Weight 0
Start of iteration 2
Current loss value: 196525.58
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_2.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_8_2/salida_8_iteration_0_0_2.png
Iter

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta: Para este punto se utilizo la foto de Sol (perro raza Lebrel Whippet) y la pintura de Pablo Piccasso Les Demoiselles d'Avignon. Se realizo el mismo analisis que el solicitado en la pregunta 7 y 8.
<img src="images\pregunta_9.jpeg">

En este punto se realizo la iteracion con los parametros establecidos en la pregunta 7; y ademas se realizó la misma iteración que en la pregunta 9 sobre los pesos de estilo y contenido. En ambos casos se observan los mismos resulatdos mencionados con anticipación.


In [30]:
#Correr esto solo para pregunta 9
base_image_path = base_image_path_2
style_reference_image_path = style_reference_image_path_2

### Primera iteracion con los parametros del comienzo

<img src="images\pregunta_9_1.jpeg">



In [ ]:
!mkdir /output_9_1
result_prefix = Path("/output_9_1")



evaluator = Evaluator()
iterations = 100
# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('sol_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Start of iteration 0
Current loss value: 16506997000.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_1/sol_iteration_0.png
Iteration 0 completed in 18s
Start of iteration 1
Current loss value: 8292506600.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_1/sol_iteration_1.png
Iteration 1 completed in 9s
Start of iteration 2
Current loss value: 5570964500.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_1/sol_iteration_2.png
Iteration 2 completed in 9s
Start of iteration 3
Current loss value: 4429279000.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_1/sol_iteration_3.png
Iteration 3 completed in 9s
Start of iteration 4
Current loss value: 3804252400.0
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_1/sol_ite

### Segunda iteración con las varaciones sobre los pesos de estilo y contenido

<img src="images\pregunta_9_2.jpeg">



In [47]:
evaluator = Evaluator()
# Creamos el directorio para los archivos de salida
!mkdir /output_9_2
result_prefix = Path("/output_9_2")



iterations =10
# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for sw in [0,1,100]:
  style_weight = sw
  for cw in [0,1,100]:
    content_weight = cw
    for i in range(iterations):
      print('Start Style Weight', sw)
      print('Start Content Weight', cw) 
      print('Start of iteration', i)
      start_time = time.time()

      loss = K.variable(0.0)
      layer_features = outputs_dict['block5_conv2']
      base_image_features = layer_features[0, :, :, :]
      combination_features = layer_features[2, :, :, :]
      loss = loss + content_weight * content_loss(base_image_features,
                                              combination_features)
      feature_layers = ['block1_conv1', 'block2_conv1',
                    'block3_conv1', 'block4_conv1',
                    'block5_conv1']
      for layer_name in feature_layers:
        layer_features = outputs_dict[layer_name]
        style_reference_features = layer_features[1, :, :, :] 
        combination_features = layer_features[2, :, :, :]
        sl = style_loss(style_reference_features, combination_features)
        loss = loss + (style_weight / len(feature_layers)) * sl
      loss = loss + total_variation_weight * total_variation_loss(combination_image)

      grads = K.gradients(loss, combination_image)
      outputs = [loss]
      if isinstance(grads, (list, tuple)):
        outputs += grads
      else:
        outputs.append(grads)
      f_outputs = K.function([combination_image], outputs)

      x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
      print('Current loss value:', min_val)
      # save current generated image
      img = deprocess_image(x.copy())
      fname = result_prefix / ('sol_iteration_%d_%d_%d.png' % (sw,cw,i))
      print('Image saved as', fname)
      save_img(fname, img)
      end_time = time.time()
      print('Image saved as', fname)
      print('Iteration %d completed in %ds' % (i, end_time - start_time))

mkdir: cannot create directory ‘/content/output/pesos/’: File exists
Start Style Weight 0
Start Content Weight 0
Start of iteration 0
Current loss value: 1823761.6
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_2/sol_iteration_0_0_0.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_2/sol_iteration_0_0_0.png
Iteration 0 completed in 13s
Start Style Weight 0
Start Content Weight 0
Start of iteration 1
Current loss value: 396829.9
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_2/sol_iteration_0_0_1.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_2/sol_iteration_0_0_1.png
Iteration 1 completed in 14s
Start Style Weight 0
Start Content Weight 0
Start of iteration 2
Current loss value: 178940.61
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/salida_9_2/sol_iteration_0_0_2.png
Image saved as /content/drive/My Drive/Colab Notebooks/Trabajo_Final/